In [132]:
import pandas as pd
import numpy as np

## SQL Implementation Workbench

I only rarely used SQL with Python before, let alone in a "non-tutorial" project. My goal for this piece is to find a more efficient storage solution for the river data, experiment with SQL technologies and hone my SQL programming skills.

Get to now Python's built in SQLlite3 module for prototyping a SQL Solution  
Migrate the prototype to a Postgres Server  
Migrate the prototype to a MySQL Server  
Migrate the prototype to a cloud based SQL Server (Azure, Google or AWS) from Python  
Migrate the prototype using Azure Data Factory  
Hone SQL Programming skills  
Explore retrival from SQL to Pandas  

#seen by https://realpython.com/python-sql-libraries/
https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany
https://www.python-lernen.de/sqlite-datenbank-auslesen.htm

In [133]:
using_sql = True

In [134]:
if using_sql == True:
    
    import sqlite3
    from sqlite3 import Error

    def create_connection(path):
        connection = None
        try:
            connection = sqlite3.connect(path)
            print("Connection to SQLite DB successful")
    
        except Error as e:
    
            print(f"The error '{e}' occurred")

        return connection

    connection = create_connection('''E:\\sm_app.sqlite''')

Connection to SQLite DB successful


Create

Manipulate

Query

In [135]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [136]:
create_master_table = """
CREATE TABLE IF NOT EXISTS master_table (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  messstelle            TEXT NOT NULL,
  gewässer              TEXT NOT NULL,
  datetime              DATETIME,
  wasserstand_cm       INT,
  änderungen_2std_cm   INT,
  abfluss_cm           INT,
  meldestufe            TINYINT,
  jährlichkeit          INT,
  vorhersage            INT
);
"""

In [137]:
execute_query(connection, create_master_table)

Query executed successfully


In [138]:
#Load the complete csv to dataframe
for file in os.listdir():
    if 'bay_river_pegel_' and 'csv' in file:
        waterlevel_hist = pd.read_csv(file, parse_dates=True, na_values= '---')
        waterlevel_hist.dropna(axis=1, how='all', inplace=True)
        break
    
    else:
        waterlevel_hist = None


In [139]:
waterlevel_hist = waterlevel_hist[waterlevel_hist['gewässer'] == 'Isar']

In [151]:
waterlevel_hist

,messstelle,gewässer,datumzeit,wasser_stand_cm_,änderungseit2std__cm_,abfluss_m__s_,melde_stufe
161,Plattling,Isar,2021-10-04 21:00:00,176.0,1,138.0,0.0
162,Landau,Isar,2021-10-04 21:00:00,109.0,0,167.0,0.0
163,Landshut Birket,Isar,2021-10-04 21:00:00,116.0,0,151.0,0.0
164,Freising,Isar,2021-10-04 21:00:00,29.0,-1,216.0,0.0
165,München,Isar,2021-10-04 21:00:00,112.0,2,691.0,0.0
...,...,...,...,...,...,...,...
262333,Landau,Isar,2021-08-23 00:00:00,121.0,1,206.0,0.0
262340,Sylvenstein,Isar,2021-08-23 00:00:00,255.0,-1,184.0,0.0
262341,Lenggries,Isar,2021-08-23 00:00:00,120.0,1,244.0,0.0
262342,Bad Tölz Brücke,Isar,2021-08-23 00:00:00,75.0,2,NaN,0.0


In [140]:
for row in waterlevel_hist.itertuples():
      create_observations = ('''
                INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
                VALUES ({}, {}, {}, {}, {}, {}, {})
                '''.format(
                row.messstelle,
                row.gewässer,
                row.datumzeit,
                row.wasser_stand_cm_,
                row.änderungseit2std__cm_,
                row.abfluss_m__s_,
                row.melde_stufe
                ))  
    
execute_query(connection, create_observations)


The error 'near "00": syntax error' occurred


In [141]:
print(repr('Mitteldachstetten,Fränkische Rezat,2021-08-23 21:00:00,113,0,010,0,---,'))

'Mitteldachstetten,Fränkische Rezat,2021-08-23 21:00:00,113,0,010,0,---,'


In [142]:
for row in waterlevel_hist[:10].itertuples():
  create_observations = ('''
    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES ({}, {}, {}, {}, {}, {}, {})
    '''.format(
        row.messstelle,
        str(row.gewässer),
        str(row.datumzeit),
        row.wasser_stand_cm_,
        row.änderungseit2std__cm_,
        row.abfluss_m__s_,
        row.melde_stufe
    ))  

  print(create_observations)
  print(type(create_observations))


    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES (Plattling, Isar, 2021-10-04 21:00:00, 176.0, 1, 138.0, 0.0)
    
<class 'str'>

    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES (Landau, Isar, 2021-10-04 21:00:00, 109.0, 0, 167.0, 0.0)
    
<class 'str'>

    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES (Landshut Birket, Isar, 2021-10-04 21:00:00, 116.0, 0, 151.0, 0.0)
    
<class 'str'>

    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES (Freising, Isar, 2021-10-04 21:00:00, 29.0, -1, 216.0, 0.0)
    
<class 'str'>

    INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe)
    VALUES (München

In [163]:
%time
import csv, sqlite3

con = sqlite3.connect("E\sm_app.sqlite") # change to 'sqlite:///your_filename.db'
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS master_table (
  messstelle            TEXT NOT NULL,
  gewässer              TEXT NOT NULL,
  datetime              DATETIME,
  wasserstand_cm       INT,
  änderungen_2std_cm   INT,
  abfluss_cm           INT,
  meldestufe            TINYINT,
  jährlichkeit          INT,
  vorhersage            INT
);""") # use your column names here

with open('bay_river_pegel_2021-08-23_bis_2021-10-04.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['messstelle'], i['gewässer'], i['datumzeit'], i['wasser_stand_cm_'], 
              i['änderungseit2std__cm_'], i['abfluss_m__s_'], i['melde_stufe'], i['jähr_lichkeit'], 
              i['vorher_sage']) for i in dr]

cur.executemany("INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe, jährlichkeit, vorhersage) VALUES (?, ?, ? ,?,?,?,?,?,?);", to_db)
con.commit()

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 8.11 µs


In [164]:
%time
import csv, sqlite3

con = sqlite3.connect("E\sm_app.sqlite") # change to 'sqlite:///your_filename.db'
cur = con.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS master_table (
  messstelle            TEXT NOT NULL,
  gewässer              TEXT NOT NULL,
  datetime              DATETIME,
  wasserstand_cm       INT,
  änderungen_2std_cm   INT,
  abfluss_cm           INT,
  meldestufe            TINYINT,
  jährlichkeit          INT,
  vorhersage            INT
);""") # use your column names here

with open('bay_river_pegel_2021-08-23_bis_2021-10-04.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['messstelle'], i['gewässer'], i['datumzeit'], i['wasser_stand_cm_'], 
              i['änderungseit2std__cm_'], i['abfluss_m__s_'], i['melde_stufe'], i['jähr_lichkeit'], 
              i['vorher_sage']) for i in dr]

cur.executemany("INSERT INTO master_table (messstelle, gewässer, datetime, wasserstand_cm, änderungen_2std_cm, abfluss_cm, meldestufe) VALUES (?, ?, ? ,?,?,?,?);", waterlevel_hist.values)
con.commit()

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


In [144]:
cur.execute("SELECT messstelle, gewässer FROM master_table LIMIT 5")
inhalt = cur.fetchall()
print(inhalt)

[('Schenkenau', 'Itz'), ('Büg', 'Schwabach'), ('Bad Vilbel', 'Nidda'), ('Arzberg', 'Röslau'), ('Hof', 'Sächsische Saale')]


In [145]:
con.close()

In [146]:
with open('bay_river_pegel_2021-08-23_bis_2021-10-04.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['messstelle'], i['gewässer'], i['datumzeit'], i['wasser_stand_cm_'], i['änderungseit2std__cm_'], i['abfluss_m__s_'], i['melde_stufe'], i['jähr_lichkeit'], i['vorher_sage']) for i in dr]

In [170]:
print(to_db[:3])
type(to_db)

[('Schenkenau', 'Itz', '2021-10-04 21:00:00', '157', '-7', '254', '0', '---', ''), ('Büg', 'Schwabach', '2021-10-04 21:00:00', '166', '1', '026', '0', '---', ''), ('Bad Vilbel', 'Nidda', '2021-10-04 21:00:00', '69', '1', '476', '0', '---', '')]


list

In [168]:
waterlevel_hist[:3].values

array([['Plattling', 'Isar', '2021-10-04 21:00:00', 176.0, 1, 138.0, 0.0],
       ['Landau', 'Isar', '2021-10-04 21:00:00', 109.0, 0, 167.0, 0.0],
       ['Landshut Birket', 'Isar', '2021-10-04 21:00:00', 116.0, 0,
        151.0, 0.0]], dtype=object)